In [1]:
import tensorflow as tf
import numpy as np

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
sequence_length = 96
embedding_length = 64

## Placeholder

In [3]:
input_data = tf.placeholder(tf.float32,[None,sequence_length,embedding_length])

## [Build RNN Cell](https://blog.csdn.net/abclhq2005/article/details/78683530)
`tf.contrib.rnn.BasicLSTMCell`與`tf.contrib.rnn.LSTMCell`不同在於新增了3個東西
1. `use_peepholes=False`：窺視孔，於2001年`Gers`提出，相較於原本有3個gate的LSTM多出了一個gate，預設為`False`，也就是普通的LSTM
2. `cell_clip=None`：`c = clip_ops.clip_by_value(c, -self._cell_clip, self._cell_clip)`，將`cell`的值限制在正負`self._cell_clip`內，可以視為一種防止梯度爆炸的方法。
3. `num_proj=None,proj_clip=None`：在cell輸出`hidden_state`後再增加一層線性變換，並限制輸出的值。

In [4]:
help(tf.contrib.rnn.BasicLSTMCell)

Help on class BasicLSTMCell in module tensorflow.python.ops.rnn_cell_impl:

class BasicLSTMCell(LayerRNNCell)
 |  Basic LSTM recurrent network cell.
 |  
 |  The implementation is based on: http://arxiv.org/abs/1409.2329.
 |  
 |  We add forget_bias (default: 1) to the biases of the forget gate in order to
 |  reduce the scale of forgetting in the beginning of the training.
 |  
 |  It does not allow cell clipping, a projection layer, and does not
 |  use peep-hole connections: it is the basic baseline.
 |  
 |  For advanced models, please use the full @{tf.nn.rnn_cell.LSTMCell}
 |  that follows.
 |  
 |  Method resolution order:
 |      BasicLSTMCell
 |      LayerRNNCell
 |      RNNCell
 |      tensorflow.python.layers.base.Layer
 |      tensorflow.python.keras.engine.base_layer.Layer
 |      tensorflow.python.training.checkpointable.base.CheckpointableBase
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, num_units, forget_bias=1.0, state_is_tuple=True, a

In [5]:
hidden_vector_size = 100

rnn_cell = tf.contrib.rnn.LSTMCell(hidden_vector_size) 

### Get batch Size from input data rather than hard coding it

In [6]:
initial_zero_h = tf.matmul(tf.reduce_mean(tf.zeros_like(input_data),2), # 依照第三維取平均, shape=(batch,96)
                           tf.zeros([sequence_length,hidden_vector_size])) # shape=(96,100)
# final shape=(batch, 100)

### Initial State with tuple of h,c
https://zhuanlan.zhihu.com/p/28919765

In [7]:
initial_state = tf.contrib.rnn.LSTMStateTuple(initial_zero_h,initial_zero_h)
# initial_state是hidden_state以及cell組成的tuple，真正用於lstm循環

In [8]:
# 也可以這樣分開寫，只是後面又需要使用tuple將兩個連在一起
# c, h = tf.contrib.rnn.LSTMStateTuple(initial_zero_h,initial_zero_h) 

### [Getting the outputs](https://blog.csdn.net/shuishou07/article/details/78551602)
https://stackoverflow.com/questions/44162432/analysis-of-the-output-from-tf-nn-dynamic-rnn-tensorflow-function

`tf.nn.dynamic_rnn`：動態計算rnn，因為不一定是每個example的`timesteps`都是一樣長，可以由參數`sequence_length`指定每個examples的有效`timestep`長度，但是前處理還是必須把每個example的`timesteps`padding成一樣長度，指定長度後對於padding的部分就不計算。

In [9]:
help(tf.nn.dynamic_rnn)

Help on function dynamic_rnn in module tensorflow.python.ops.rnn:

dynamic_rnn(cell, inputs, sequence_length=None, initial_state=None, dtype=None, parallel_iterations=None, swap_memory=False, time_major=False, scope=None)
    Creates a recurrent neural network specified by RNNCell `cell`.
    
    Performs fully dynamic unrolling of `inputs`.
    
    Example:
    
    ```python
    # create a BasicRNNCell
    rnn_cell = tf.nn.rnn_cell.BasicRNNCell(hidden_size)
    
    # 'outputs' is a tensor of shape [batch_size, max_time, cell_state_size]
    
    # defining initial state
    initial_state = rnn_cell.zero_state(batch_size, dtype=tf.float32)
    
    # 'state' is a tensor of shape [batch_size, cell_state_size]
    outputs, state = tf.nn.dynamic_rnn(rnn_cell, input_data,
                                       initial_state=initial_state,
                                       dtype=tf.float32)
    ```
    
    ```python
    # create 2 LSTMCells
    rnn_layers = [tf.nn.rnn_cell.LSTMCel

In [10]:
outputs, state = tf.nn.dynamic_rnn(rnn_cell, input_data, 
                                   initial_state=initial_state,
                                   dtype=tf.float32) # sequence_length預設為None，也就是不需要使用tf.nn.dynamic

`tf.nn.dynamic_rnn`的輸出分成兩部分：
* 第一部分`outputs`的shape為`(batch,timesteps,hidden_vector_size)`，也就是`input`的shape`(batch,timesteps,features)`經過計算後得到`(batch, timesteps, hidden_vector_size)`，輸入時每一步`timestep`都會由`features`計算為`hidden_vector_size`，如果只在意最後一步`timesteps`的輸出，只要指定即可，即`outputs[:,-1,:]`。

In [11]:
outputs #shape(batch,96,100), 每步timestep的輸出

<tf.Tensor 'rnn/transpose_1:0' shape=(?, 96, 100) dtype=float32>

In [12]:
state # 由最後一步timestep中的(cell,hidden_state)組成的tuple

LSTMStateTuple(c=<tf.Tensor 'rnn/while/Exit_3:0' shape=(?, 100) dtype=float32>, h=<tf.Tensor 'rnn/while/Exit_4:0' shape=(?, 100) dtype=float32>)

In [13]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

### Use any batch Size

In [14]:
batch_size = 14
fake_input = np.random.uniform(size=[batch_size,96,64])
outputs1 = outputs.eval(session=sess,feed_dict={input_data:fake_input}) # sess.run另外一種執行方式
print(outputs1.shape) #(batch, timesteps, hidden_vector_size)
outputs1[:,-1,:] # 最後一步timestep輸出

(14, 96, 100)


array([[ 0.06776507, -0.16604237,  0.29889047, ..., -0.2910829 ,
        -0.02830626,  0.17666036],
       [ 0.0161004 , -0.11812118,  0.28954205, ..., -0.2820639 ,
        -0.10455608,  0.19264744],
       [ 0.06198378, -0.1680148 ,  0.2578251 , ..., -0.2776524 ,
         0.02449181,  0.2434168 ],
       ...,
       [ 0.01514441, -0.06312319,  0.22368349, ..., -0.22516553,
        -0.06407987,  0.21258672],
       [ 0.03427761, -0.07078353,  0.30546343, ..., -0.26227936,
        -0.10738929,  0.28163087],
       [ 0.05872219, -0.13179702,  0.23218733, ..., -0.32039693,
        -0.14472461,  0.201169  ]], dtype=float32)

In [15]:
state1 = sess.run(state, feed_dict={input_data:fake_input})

In [16]:
print(state1.c.shape) # 最後一步timestep的cell值
state1.c

(14, 100)


array([[ 0.14182168, -0.38372934,  0.65008837, ..., -0.60773027,
        -0.04967897,  0.4872932 ],
       [ 0.03204466, -0.2811767 ,  0.6193227 , ..., -0.5679249 ,
        -0.18388133,  0.524578  ],
       [ 0.138229  , -0.37704027,  0.523069  , ..., -0.62181395,
         0.04411354,  0.6877756 ],
       ...,
       [ 0.03829008, -0.16699168,  0.4415437 , ..., -0.53708845,
        -0.11180121,  0.5098827 ],
       [ 0.07112903, -0.16521525,  0.704515  , ..., -0.5881021 ,
        -0.19752544,  0.9376542 ],
       [ 0.12712282, -0.3374092 ,  0.4719813 , ..., -0.7883067 ,
        -0.233245  ,  0.57803166]], dtype=float32)

In [17]:
print(state1.h.shape)  # 最後一步timestep的hidden_stat值, 最後都是由這個hidden_state搭配dense去作為真正輸出
state1.h

(14, 100)


array([[ 0.06776507, -0.16604237,  0.29889047, ..., -0.2910829 ,
        -0.02830626,  0.17666036],
       [ 0.0161004 , -0.11812118,  0.28954205, ..., -0.2820639 ,
        -0.10455608,  0.19264744],
       [ 0.06198378, -0.1680148 ,  0.2578251 , ..., -0.2776524 ,
         0.02449181,  0.2434168 ],
       ...,
       [ 0.01514441, -0.06312319,  0.22368349, ..., -0.22516553,
        -0.06407987,  0.21258672],
       [ 0.03427761, -0.07078353,  0.30546343, ..., -0.26227936,
        -0.10738929,  0.28163087],
       [ 0.05872219, -0.13179702,  0.23218733, ..., -0.32039693,
        -0.14472461,  0.201169  ]], dtype=float32)